## Preparação do banco de dados MySQL

### Coleta e tratamento dos dados

Neste primeiro passo do desafio, temos que realizar uma coleta de dados por uma API fornecida pela IGTI.

A URL para se realizar a coleta é: https://api-coleta-dados.herokuapp.com/api/pessoas.

In [166]:
import json
import requests

In [167]:
#Requisitando os dados da API
url = "https://api-coleta-dados.herokuapp.com/api/pessoas"
response = requests.get(url)

In [168]:
#Status da requisição. 200: sucesso.
response.status_code

200

In [169]:
#Checando o tipo de dados do conteúdo requisitado
print(type(response.content))

<class 'bytes'>


In [170]:
#Convertendo os dados de conteúdo para JSON
dados = json.loads(response.content) #serializando e transformando em JSON
print(type(dados))

<class 'list'>


In [171]:
#Visualizando os cinco primeiros registros da lista de dados
#Vemos que os dados estão em forma de lista de dicionários
dados[:5]

[{'nome': 'Silvana Hadassa',
  'idade': None,
  'data_nasc': '12/06/1964',
  'sexo': 'Feminino',
  'signo': 'Gêmeos',
  'cidade': 'Brasília',
  'estado': 'DF',
  'altura': '1,67',
  'peso': 50,
  'tipo_sanguineo': 'A+',
  'cor': 'vermelho'},
 {'nome': 'Thales Kaique',
  'idade': 27,
  'data_nasc': '10/12/1993',
  'sexo': 'Masculino',
  'signo': 'Sagitário',
  'cidade': 'Campo Grande',
  'estado': 'MS',
  'altura': '1,63',
  'peso': 110,
  'tipo_sanguineo': 'B+',
  'cor': 'laranja'},
 {'nome': 'Henry Nathan',
  'idade': 29,
  'data_nasc': '21/09/1991',
  'sexo': 'Masculino',
  'signo': 'Virgem',
  'cidade': 'Macapá',
  'estado': 'AP',
  'altura': '1,86',
  'peso': 106,
  'tipo_sanguineo': 'AB-',
  'cor': 'verde'},
 {'nome': 'Lorenzo Vinicius',
  'idade': 64,
  'data_nasc': '22/10/1956',
  'sexo': 'Masculino',
  'signo': 'Libra',
  'cidade': 'Contagem',
  'estado': 'MG',
  'altura': '1,87',
  'peso': 100,
  'tipo_sanguineo': 'O+',
  'cor': 'vermelho'},
 {'nome': 'Cauã Henrique',
  'idade

#### Tratamento de dados
Foram solicitados o preenchimento do campo `'idade'` quando este estiver nulo, e o preenchimento do ano de nascimento quando este estiver ausente no campo `'data_nasc'`.
Para tais tratamentos, consideraremos a data atual como sendo **20/12/2020**, que é a data de entrega do presente Desafio.

In [172]:
#Preenchendo o campo 'idade', quando nulo
for pessoa in dados:
    if pessoa['idade'] is None:
        dia_nasc = int(pessoa['data_nasc'][:2])
        mes_nasc = int(pessoa['data_nasc'][3:5])
        ano_nasc = int(pessoa['data_nasc'][6:])
        idade = 2020-ano_nasc
        if (mes_nasc == 12) and (dia_nasc > 20):
            idade -= 1
        pessoa['idade'] = idade

In [173]:
#Preenchendo o ano de nascimento no campo 'data_nasc', quando ausente
for pessoa in dados:
    if len(pessoa['data_nasc']) < 10:
        dia_nasc = int(pessoa['data_nasc'][:2])
        mes_nasc = int(pessoa['data_nasc'][3:5])
        idade = pessoa['idade']
        ano_nasc = 2020-idade
        if (mes_nasc == 12) and (dia_nasc > 20):
            ano_nasc -= 1
        string = '/%s'%ano_nasc
        pessoa['data_nasc'] += string

In [174]:
dados[:5]

[{'nome': 'Silvana Hadassa',
  'idade': 56,
  'data_nasc': '12/06/1964',
  'sexo': 'Feminino',
  'signo': 'Gêmeos',
  'cidade': 'Brasília',
  'estado': 'DF',
  'altura': '1,67',
  'peso': 50,
  'tipo_sanguineo': 'A+',
  'cor': 'vermelho'},
 {'nome': 'Thales Kaique',
  'idade': 27,
  'data_nasc': '10/12/1993',
  'sexo': 'Masculino',
  'signo': 'Sagitário',
  'cidade': 'Campo Grande',
  'estado': 'MS',
  'altura': '1,63',
  'peso': 110,
  'tipo_sanguineo': 'B+',
  'cor': 'laranja'},
 {'nome': 'Henry Nathan',
  'idade': 29,
  'data_nasc': '21/09/1991',
  'sexo': 'Masculino',
  'signo': 'Virgem',
  'cidade': 'Macapá',
  'estado': 'AP',
  'altura': '1,86',
  'peso': 106,
  'tipo_sanguineo': 'AB-',
  'cor': 'verde'},
 {'nome': 'Lorenzo Vinicius',
  'idade': 64,
  'data_nasc': '22/10/1956',
  'sexo': 'Masculino',
  'signo': 'Libra',
  'cidade': 'Contagem',
  'estado': 'MG',
  'altura': '1,87',
  'peso': 100,
  'tipo_sanguineo': 'O+',
  'cor': 'vermelho'},
 {'nome': 'Cauã Henrique',
  'idade':

### Alimentação do *database* MySQL `db_desafio`

Os dados coletados na API disponibilizada serão alimentados ao *database* `db_desafio` (query para criação em `criacao_db_desafio_MySQL.sql`).

O **EER** deste banco de dados é:
<img src='der.png'>

In [175]:
#Importando a biblioteca mysql.connector e datetime
import mysql.connector
from datetime import datetime

In [176]:
#Conectando-se ao banco de dados
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='igti',
    database='db_desafio')

mycursor = mydb.cursor()

In [177]:
#Inserir dados na tabela tb_cor
for pessoa in dados:
    cor = pessoa['cor']

    query = "INSERT INTO tb_cor (cor) SELECT * FROM (SELECT '{}') AS tmp ".format(cor)
    query += "WHERE NOT EXISTS (SELECT cor FROM tb_cor WHERE cor = '{}') LIMIT 1".format(cor)
    fquery = query.format(cor, cor)
    
    mycursor.execute(query) #executando a query
    mydb.commit() #confirmando a inserção

In [178]:
#Inserir dados na tabela tb_tiposanguineo
for pessoa in dados:
    tipo_sanguineo = pessoa['tipo_sanguineo']

    query = "INSERT INTO tb_tiposanguineo (tipo) SELECT * FROM (SELECT '{}') AS tmp ".format(tipo_sanguineo)
    query += "WHERE NOT EXISTS (SELECT tipo FROM tb_tiposanguineo WHERE tipo = '{}') LIMIT 1".format(tipo_sanguineo)
    fquery = query.format(tipo_sanguineo, tipo_sanguineo)

    mycursor.execute(query) #executando a query
    mydb.commit() #confirmando a inserção

In [179]:
#Inserir dados na tabela tb_estado
for pessoa in dados:
    estado = pessoa['estado']

    query = "INSERT INTO tb_estado (sigla) SELECT * FROM (SELECT '{}') AS tmp "
    query += "WHERE NOT EXISTS (SELECT sigla FROM tb_estado WHERE sigla = '{}') LIMIT 1"
    fquery = query.format(estado, estado)

    mycursor.execute(fquery) #executando a query
    mydb.commit() #confirmando a inserção

In [180]:
#Inserir dados na tabela tb_cidade
for pessoa in dados:
    cidade = pessoa['cidade']
    estado = pessoa['estado']
    
    query = "INSERT INTO tb_cidade (nome, id_estado) "
    query += "SELECT * FROM (SELECT '{}', (SELECT id FROM tb_estado WHERE sigla = '{}')) AS tmp "
    query += "WHERE NOT EXISTS (SELECT nome FROM tb_cidade WHERE nome = '{}') LIMIT 1"
    fquery = query.format(cidade, estado, cidade)

    mycursor.execute(fquery) #executando a query
    mydb.commit() #confirmando a inserção

In [181]:
#Inserir dados na tabela tb_pessoa
for pessoa in dados:
    nome = pessoa['nome']
    idade = int(pessoa['idade'])
    data_nasc = pessoa['data_nasc']
    sexo = pessoa['sexo'][0:1]
    signo = pessoa['signo']
    altura = pessoa['altura'].replace(',', '.')
    peso = pessoa['peso']
    cidade = pessoa['cidade']
    cor = pessoa['cor']
    tipo_sanguineo = pessoa['tipo_sanguineo']
    
    query = "INSERT INTO tb_pessoa (nome, idade, data_nasc, sexo, signo, altura, peso, id_cidade, id_cor, id_tiposanguineo) "
    query += "VALUES ('{}', '{}', str_to_date('{}','%d/%m/%Y'), '{}', '{}', '{}', '{}', "
    query += "(SELECT id FROM tb_cidade WHERE nome = '{}'), (SELECT id FROM tb_cor WHERE cor = '{}'), "
    query += "(SELECT id FROM tb_tiposanguineo WHERE tipo = '{}'))"
    fquery = query.format(nome, idade, data_nasc, sexo, signo, altura, peso, cidade, cor, tipo_sanguineo)

    mycursor.execute(fquery) #executando a query
    mydb.commit() #confirmando a inserção

In [182]:
#Encerrando o cursor e a conexão
mycursor.close()
mydb.close()